**Land Ownership Cleaning**

In [ ]:
%run ../bootstrap.py
setup_project_path()

from scripts.io_helpers import read_raw_layer, export_interim
from scripts.data_helpers import to_buffer_crs

Let's load and take a look at the data.

In [2]:
gdf = read_raw_layer("land_ownership_raw")

print("Columns:", gdf.columns.tolist())
print("State names: ", gdf['State_Nm'].unique())

/Users/loganproffitt/Desktop/CampGIS.nosync/Repo/CampGIS/venv/lib/python3.11/site-packages/pyogrio/geopandas.py:275: UserWarning: More than one layer found in 'PADUS4_1_StateCO.gdb': 'PADUS4_1Fee_State_CO' (default), 'PADUS4_1Easement_State_CO', 'PADUS4_1Proclamation_State_CO', 'PADUS4_1Comb_DOD_Trib_NGP_Fee_Desig_Ease_State_CO', 'PADUS4_1Designation_State_CO', 'Public_Access', 'Agency_Name', 'Agency_Type', 'Category', 'Designation_Type', 'GAP_Status', 'IUCN_Category', 'State_Name'. Specify layer parameter to avoid this warning.
  result = read_func(
/Users/loganproffitt/Desktop/CampGIS.nosync/Repo/CampGIS/venv/lib/python3.11/site-packages/pyogrio/raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_r

Columns: ['Category', 'Own_Type', 'Own_Name', 'Loc_Own', 'Mang_Type', 'Mang_Name', 'Loc_Mang', 'Des_Tp', 'Loc_Ds', 'Unit_Nm', 'Loc_Nm', 'State_Nm', 'Agg_Src', 'GIS_Src', 'Src_Date', 'GIS_Acres', 'Source_PAID', 'WDPA_Cd', 'Pub_Access', 'Access_Src', 'Access_Dt', 'GAP_Sts', 'GAPCdSrc', 'GAPCdDt', 'IUCN_Cat', 'IUCNCtSrc', 'IUCNCtDt', 'Date_Est', 'Comments', 'd_Category', 'd_Own_Type', 'd_Own_Name', 'd_Mang_Type', 'd_Mang_Name', 'd_Des_Tp', 'd_State_Nm', 'd_Pub_Access', 'd_GAP_Sts', 'd_IUCN_Cat', 'Shape_Length', 'Shape_Area', 'geometry']
State names:  ['CO']


Data is already isolated to CO — if we want other states in the future, we can link to the full U.S. dataset and select desired states.

Let's filter down to useful columns.

In [3]:
columns_keep = [
    "Mang_Name",
    "Loc_Mang",
    "Unit_Nm",
    "Loc_Nm",
    "Des_Tp",
    "Pub_Access",
    "Own_Name",
    "State_Nm",
    'Shape_Length', 
    'Shape_Area',
    "geometry"
]

gdf_cols = gdf[columns_keep]
gdf_cols.head()


,Mang_Name,Loc_Mang,Unit_Nm,Loc_Nm,Des_Tp,Pub_Access,Own_Name,State_Nm,Shape_Length,Shape_Area,geometry
0,ARS,Agricultural Research Service (ARS),Central Plains Experimental Range,Central Plains Experimental Range,FOTH,OA,UNK,CO,8.034916e+04,5.938316e+07,"MULTIPOLYGON (((-728264.011 2009738.467, -7279..."
1,DOE,Department of Energy (DOE),Federal Land,,FOTH,XA,UNK,CO,4.644972e+03,8.304424e+05,"MULTIPOLYGON (((-1000907.923 1906834.96, -1000..."
2,BLM,BLM,Colorado River Valley Field Office,Bureau of Land Management,PUB,OA,BLM,CO,3.096733e+06,2.309653e+09,"MULTIPOLYGON (((-923075.889 1850198.558, -9231..."
3,BLM,BLM,Grand Junction Field Office,Bureau of Land Management,PUB,OA,BLM,CO,3.832750e+06,5.162145e+09,"MULTIPOLYGON (((-1120162.991 1792729.163, -112..."
4,BLM,BLM,Gunnison Field Office,Bureau of Land Management,PUB,OA,BLM,CO,3.977402e+06,2.668452e+09,"MULTIPOLYGON (((-1011674.921 1697854.958, -101..."


Now, we want to filter on relevant land managers. Even if a piece of land is owned by a relevant owner, management determines local camping legality, so we'll filter on manager name.

In [4]:
print("Manager names: \n", gdf_cols['Mang_Name'].unique())

Manager names: 
 ['ARS' 'DOE' 'BLM' 'USBR' 'SPR' 'USACE' 'USFS' 'NPS' 'FWS' 'NGO' 'CITY'
 'CNTY' 'PVT' 'SFW' 'UNK' 'REG' 'OTHF' 'JNT' 'OTHS' 'UNKL' 'SLB' 'RWD'
 'DOD']


From this list, we want to keep BLM, USFS, NPS, and CNTY.

In [5]:
managers_keep = [
    'BLM',
    'USFS',
    'NPS',
    'CNTY'
]

gdf_filtered = gdf_cols[gdf_cols['Mang_Name'].isin(managers_keep)]
gdf_filtered.head()

,Mang_Name,Loc_Mang,Unit_Nm,Loc_Nm,Des_Tp,Pub_Access,Own_Name,State_Nm,Shape_Length,Shape_Area,geometry
2,BLM,BLM,Colorado River Valley Field Office,Bureau of Land Management,PUB,OA,BLM,CO,3.096733e+06,2.309653e+09,"MULTIPOLYGON (((-923075.889 1850198.558, -9231..."
3,BLM,BLM,Grand Junction Field Office,Bureau of Land Management,PUB,OA,BLM,CO,3.832750e+06,5.162145e+09,"MULTIPOLYGON (((-1120162.991 1792729.163, -112..."
4,BLM,BLM,Gunnison Field Office,Bureau of Land Management,PUB,OA,BLM,CO,3.977402e+06,2.668452e+09,"MULTIPOLYGON (((-1011674.921 1697854.958, -101..."
5,BLM,BLM,Kremmling Field Office,Bureau of Land Management,PUB,OA,BLM,CO,2.651089e+06,1.527993e+09,"MULTIPOLYGON (((-839688.745 1921367.61, -83972..."
6,BLM,BLM,Little Snake Field Office,Bureau of Land Management,PUB,OA,BLM,CO,5.976242e+06,5.414507e+09,"MULTIPOLYGON (((-911540.913 1948621.217, -9115..."


Let's make sure all geometries have the same type:

In [6]:
print("Geometry types:", gdf_filtered.geom_type.unique())

Geometry types: ['MultiPolygon']


All good. Let's project to the common CRS for future intersections.

In [7]:
gdf_filtered = to_buffer_crs(gdf_filtered)

Finally, let's fix invalid shapes.

In [8]:
# Identify invalid shapes
invalid = ~gdf_filtered.is_valid
print(f"Invalid geometries: {invalid.sum()}")

# Only fix those
gdf_filtered.loc[invalid, "geometry"] = gdf_filtered.loc[invalid, "geometry"].buffer(0)


Invalid geometries: 15


Ready to export!

In [9]:
export_interim(gdf_filtered, "land_ownership_clean", driver="GPKG", verbose=True)

Saved to interim: /Users/loganproffitt/Desktop/CampGIS.nosync/Repo/CampGIS/data/interim/land_ownership_clean.gpkg
